In [1]:
from google.colab import drive
drive.mount("/content/drive")

basicpath = "/content/drive/MyDrive/졸프"

import pandas as pd
import numpy as np
import os
import json

Mounted at /content/drive


# 실제 review가 적힌 product를 파악하기 위한 과정

In [2]:
raw_review_data_file = "Movies_and_TV.json"

**attribute의 의미 및 종류 / 그리고 삭제할 attribute 선정 및 기준 (preprocessing)**

유저의 아이템에 대한 선호도를 바탕으로 collaborating filtering 방식으로, matrix factorization 을 사용할 것이기 때문에 
기본적으로는 **userid, itemid, rating** 만 반영되면 된다.

---

**overall : 평점을 의미** -> rating

**reviewerID : 리뷰를 남긴 사람을 의미** -> itemid

**asin : 물건의 고유번호** -> userid

In [3]:
output = open(os.path.join(basicpath, 'rating_data_delete_not_need_attribute.csv'), 'w', encoding='utf-8')
with open(os.path.join(basicpath, raw_review_data_file), "r", encoding="utf-8") as json_file:
    json_string = json_file.readline()
    while json_string:
        data = json.loads(json_string)
        output.write(str(data['reviewerID']) + ',')
        output.write(str(data['asin']) + ',')
        output.write(str(data['overall']) + '\n')
        json_string = json_file.readline()
output.close()

FileNotFoundError: ignored

processing 한 파일이 잘 만들어졌는지 확인

In [4]:
rating_process_file = 'rating_data_delete_not_need_attribute.csv'
rating_process_data = pd.read_csv(os.path.join(basicpath, rating_process_file), delimiter=',', names=['user_id', 'item_id', 'rating'])
rating_process_data.head(10)

FileNotFoundError: ignored

In [ ]:
rating_process_data

,user_id,item_id,rating
0,A3478QRKQDOPQ2,0001527665,5.0
1,A2VHSG6TZHU1OB,0001527665,5.0
2,A23EJWOW1TLENE,0001527665,5.0
3,A1KM9FNEJ8Q171,0001527665,5.0
4,A38LY2SSHVHRYB,0001527665,4.0
...,...,...,...
8765563,A29XKBB6SEYTN6,B01HJ6R77G,5.0
8765564,ASEBX8TBYWQWA,B01HJ6R77G,4.0
8765565,AK4XZBLSY1WC9,B01HJCCLOY,1.0
8765566,AK19IXB9I4175,B01HJCCLOY,3.0


In [ ]:
rating_process_data.isnull().sum()

user_id    0
item_id    0
rating     0
dtype: int64

In [ ]:
rating_process_data.duplicated().sum()

254789

In [ ]:
drop_duplicate_data = rating_process_data.drop_duplicates()
drop_duplicate_data = drop_duplicate_data.reset_index()
del drop_duplicate_data['index']
drop_duplicate_data

,user_id,item_id,rating
0,A3478QRKQDOPQ2,0001527665,5.0
1,A2VHSG6TZHU1OB,0001527665,5.0
2,A23EJWOW1TLENE,0001527665,5.0
3,A1KM9FNEJ8Q171,0001527665,5.0
4,A38LY2SSHVHRYB,0001527665,4.0
...,...,...,...
8510774,A29XKBB6SEYTN6,B01HJ6R77G,5.0
8510775,ASEBX8TBYWQWA,B01HJ6R77G,4.0
8510776,AK4XZBLSY1WC9,B01HJCCLOY,1.0
8510777,AK19IXB9I4175,B01HJCCLOY,3.0


In [ ]:
drop_duplicate_data['category_item_id'] = drop_duplicate_data['item_id'].astype("category")
drop_duplicate_data['category_item_id']

0          0001527665
1          0001527665
2          0001527665
3          0001527665
4          0001527665
              ...    
8510774    B01HJ6R77G
8510775    B01HJ6R77G
8510776    B01HJCCLOY
8510777    B01HJCCLOY
8510778    B01HJF79XO
Name: category_item_id, Length: 8510779, dtype: category
Categories (182032, object): ['0000143502', '0000143529', '0000143561', '0000143588', ...,
                              'B01HJ3E0PQ', 'B01HJ6R77G', 'B01HJCCLOY', 'B01HJF79XO']

In [ ]:
drop_duplicate_data

,user_id,item_id,rating,category_item_id
0,A3478QRKQDOPQ2,0001527665,5.0,0001527665
1,A2VHSG6TZHU1OB,0001527665,5.0,0001527665
2,A23EJWOW1TLENE,0001527665,5.0,0001527665
3,A1KM9FNEJ8Q171,0001527665,5.0,0001527665
4,A38LY2SSHVHRYB,0001527665,4.0,0001527665
...,...,...,...,...
8510774,A29XKBB6SEYTN6,B01HJ6R77G,5.0,B01HJ6R77G
8510775,ASEBX8TBYWQWA,B01HJ6R77G,4.0,B01HJ6R77G
8510776,AK4XZBLSY1WC9,B01HJCCLOY,1.0,B01HJCCLOY
8510777,AK19IXB9I4175,B01HJCCLOY,3.0,B01HJCCLOY


0          0001527665
1          0001527665
2          0001527665
3          0001527665
4          0001527665
              ...    
8765563    B01HJ6R77G
8765564    B01HJ6R77G
8765565    B01HJCCLOY
8765566    B01HJCCLOY
8765567    B01HJF79XO
Name: category_item_id, Length: 8765568, dtype: category
Categories (182032, object): ['0000143502', '0000143529', '0000143561', '0000143588', ...,
                              'B01HJ3E0PQ', 'B01HJ6R77G', 'B01HJCCLOY', 'B01HJF79XO']

라는 결과를 보니, 리뷰가 작성된 물건의 종류의 수는 182032 개 이다.

In [ ]:
drop_duplicate_data = drop_duplicate_data.drop(columns = 'category_item_id')

In [ ]:
drop_duplicate_data

,user_id,item_id,rating
0,A3478QRKQDOPQ2,0001527665,5.0
1,A2VHSG6TZHU1OB,0001527665,5.0
2,A23EJWOW1TLENE,0001527665,5.0
3,A1KM9FNEJ8Q171,0001527665,5.0
4,A38LY2SSHVHRYB,0001527665,4.0
...,...,...,...
8510774,A29XKBB6SEYTN6,B01HJ6R77G,5.0
8510775,ASEBX8TBYWQWA,B01HJ6R77G,4.0
8510776,AK4XZBLSY1WC9,B01HJCCLOY,1.0
8510777,AK19IXB9I4175,B01HJCCLOY,3.0


In [ ]:
item_sort = drop_duplicate_data.sort_values('item_id')
item_sort = item_sort.reset_index()
del item_sort['index']
item_sort

,user_id,item_id,rating
0,A3R5OBKS7OM2IR,0000143502,5.0
1,A3R5OBKS7OM2IR,0000143529,5.0
2,AAMEXFSK24BRC,0000143561,5.0
3,AH3QC2PC1VTGP,0000143561,2.0
4,A22PPSWLQOPWNK,0000143588,5.0
...,...,...,...
8510774,A29XKBB6SEYTN6,B01HJ6R77G,5.0
8510775,ASEBX8TBYWQWA,B01HJ6R77G,4.0
8510776,AK19IXB9I4175,B01HJCCLOY,3.0
8510777,AK4XZBLSY1WC9,B01HJCCLOY,1.0


In [ ]:
item_sort.to_csv(os.path.join(basicpath, 'rating_data_item_sort.csv'), header= None)

In [ ]:
rating_item_sort_file = 'rating_data_item_sort.csv'
rating_item_sort_data = pd.read_csv(os.path.join(basicpath, rating_item_sort_file), names=['user_id', 'item_id', 'rating'])
rating_item_sort_data

,user_id,item_id,rating
0,A3R5OBKS7OM2IR,0000143502,5.0
1,A3R5OBKS7OM2IR,0000143529,5.0
2,AAMEXFSK24BRC,0000143561,5.0
3,AH3QC2PC1VTGP,0000143561,2.0
4,A22PPSWLQOPWNK,0000143588,5.0
...,...,...,...
8510774,A29XKBB6SEYTN6,B01HJ6R77G,5.0
8510775,ASEBX8TBYWQWA,B01HJ6R77G,4.0
8510776,AK19IXB9I4175,B01HJCCLOY,3.0
8510777,AK4XZBLSY1WC9,B01HJCCLOY,1.0


In [ ]:
rating_item_sort_data.duplicated().sum()

0

In [ ]:
meta_sort_file = 'meta_data_sort.csv'
meta_sort_data = pd.read_csv(os.path.join(basicpath, meta_sort_file), names=['item_id', 'title', 'brand', 'category', 'imageURL', 'imageURLHighRes'])
meta_sort_data

,item_id,title,brand,category,imageURL,imageURLHighRes
0,0000143502,Rise and Swine (Good Eats Vol. 7),Alton Brown,"['Movies & TV', 'Movies']",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1,0000143529,My Fair Pastry (Good Eats Vol. 9),Alton Brown,"['Movies & TV', 'Movies']",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
2,0020110006,The Secrets of Divine Strength And Youthfulnes...,Joseph Prince,"['Movies & TV', 'Genre for Featured Categories...",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
3,0384948251,Ranger Ready Army Ranger Training Documentary,Learn more,"['Movies & TV', 'Movies']",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
4,0578002019,"Let's Go Guang! Chinese for Children, Vol. 1 -...",Yvonne Kong,"['Movies & TV', 'Genre for Featured Categories...",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
...,...,...,...,...,...,...
21846,B01HIQ3TO2,Barbie: Star Light Adventure,Erica Lindbeck,"['Movies & TV', 'Blu-ray', 'Movies']",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
21847,B01HIQHDAS,The Rebbe's Nigunim,Benny Friedman,"['Movies & TV', 'Movies']",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
21848,B01HJ1BKUQ,John Denver: Greatest Hits Live (Collector's E...,John Denver,"['Movies & TV', 'Music Artists', 'Denver, John']",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
21849,B01HJ1INB0,Verdi: Otello,Sonya Yoncheva,"['Movies & TV', 'Studio Specials', 'Sony Pictu...",['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...


In [ ]:
rating_table = rating_process_data.pivot_table('rating', index='user_id', columns='item_id')
rating_table

테이블을 만드는 과정에서 램초과로 안만들어졌다. 어떻게해야할까?

# ratings_only 파일 읽고 필요없는 time_stamp 부분을 삭제하는 preprocessing 진행 
(이 데이터는 이제 활용안할듯)

In [ ]:
rating_data_file = "Books.csv"

rating_data = pd.read_csv(os.path.join(basicpath, rating_data_file), delimiter=',', names=['item_id', 'user_id', 'rating', 'time_stamp'])
del rating_data['time_stamp']

rating_data.head(10)

,user_id,item_id,rating
0,0001713353,A1C6M8LCIX4M6M,5.0
1,0001713353,A1REUF3A1YCPHM,5.0
2,0001713353,A1YRBRK2XM5D5,5.0
3,0001713353,A1V8ZR5P78P4ZU,5.0
4,0001713353,A2ZB06582NXCIV,5.0
5,0001713353,ACPQVNRD3Z09X,5.0
6,0001713353,AVP0HXC9FG790,5.0
7,0001713353,A32MQTLQQN44WW,5.0
8,0001713353,A13CHIJPFCEP2M,5.0
9,0001713353,A324TTUBKTN73A,5.0


In [ ]:
rating_data.to_csv(os.path.join(basicpath, 'rating_data_delete_timestamp.csv'), index=False, header=False)

NameError: ignored

In [ ]:
rating_process_file = "rating_data_delete_timestamp.csv"

rating_process_data = pd.read_csv(os.path.join(basicpath, rating_process_file), delimiter=',', names=['user_id', 'item_id', 'rating'])

rating_process_data.head(10)

,user_id,item_id,rating
0,0001713353,A1C6M8LCIX4M6M,5.0
1,0001713353,A1REUF3A1YCPHM,5.0
2,0001713353,A1YRBRK2XM5D5,5.0
3,0001713353,A1V8ZR5P78P4ZU,5.0
4,0001713353,A2ZB06582NXCIV,5.0
5,0001713353,ACPQVNRD3Z09X,5.0
6,0001713353,AVP0HXC9FG790,5.0
7,0001713353,A32MQTLQQN44WW,5.0
8,0001713353,A13CHIJPFCEP2M,5.0
9,0001713353,A324TTUBKTN73A,5.0


In [ ]:
rating_process_data.isnull().sum()

user_id    0
item_id    0
rating     0
dtype: int64